In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../../src/"))
import plot.viz_sequence as viz_sequence
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tqdm
tqdm.tqdm_notebook()

In [ ]:
# Path to SHAP scores and TF-MoDISco results
base_path = "/users/amtseng/tfmodisco/results/"
shap_scores_path = os.path.join(base_path, "importance_scores/singletask_profile/GABPA_singletask_profile_fold9/task_7/GABPA_singletask_profile_task7_fold9_imp_scores.h5")
tfm_results_path = os.path.join(base_path, "tfmodisco/singletask_profile/GABPA_singletask_profile_fold9/task_7/GABPA_singletask_profile_task7_fold9_count_tfm.h5")
hyp_score_key = "count_hyp_scores"

In [ ]:
with h5py.File(shap_scores_path, "r") as f:
    hyp_scores = f[hyp_score_key]
    input_seqs = f["input_seqs"]
    for index in np.random.choice(hyp_scores.shape[0], size=5, replace=False):
        act_scores = hyp_scores[index] * input_seqs[index]
        plt.figure(figsize=(20, 2))
        plt.plot(np.sum(act_scores, axis=1))
        plt.show()
        viz_sequence.plot_weights(act_scores[960 : 1160], subticks_frequency=100)

In [ ]:
with h5py.File(tfm_results_path, "r") as f:
    metaclusters = f["metacluster_idx_to_submetacluster_results"]
    num_metaclusters = len(metaclusters.keys())
    for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
        metacluster = metaclusters[metacluster_key]
        print("Metacluster: %s (%d/%d)" % (metacluster_key, metacluster_i + 1, num_metaclusters))
        print("==========================================")
        if "patterns" not in metacluster["seqlets_to_patterns_result"].keys():
            continue
        patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
        num_patterns = len(patterns["all_pattern_names"][:])
        for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
            pattern_name = pattern_name.decode()
            pattern = patterns[pattern_name]
            seqlets = pattern["seqlets_and_alnmts"]["seqlets"]

            print("Pattern: %s (%d/%d)" % (pattern_name, pattern_i + 1, num_patterns))
            print("--------------------------------------")

            print("%d seqlets" % len(seqlets))
            print("Sequence")
            viz_sequence.plot_weights(pattern["sequence"]["fwd"][:])
            print("Hypothetical contributions")
            viz_sequence.plot_weights(pattern["task0_hypothetical_contribs"]["fwd"][:])
            print("Contribution_scores")
            viz_sequence.plot_weights(pattern["task0_contrib_scores"]["fwd"][:])

            pwm = pattern["sequence"]["fwd"][:]
            act_contribs = pattern["task0_contrib_scores"]["fwd"][:]